In [23]:
import numpy as np
from sklearn.svm import SVC
import tarfile
import os
import pickle

In [24]:
cifarDir = "cifar10dataset"
tarfile.open("cifar-10-python.tar.gz", "r:gz").extractall(cifarDir) # Using tarfile to extract data
print("Files extracted. Check to be sure")

Files extracted. Check to be sure


In [25]:
X = []
Y = [] # Creating the blank lists
X
# Need to extract all the batches with pickle
cifarDirBatches = os.path.join(cifarDir,"cifar-10-batches-py")
for i in range(1, 6): # data_batch_1 to data_batch_5
    f = os.path.join(cifarDirBatches, f"data_batch_{i}") # Open each batch iteratively
    batch = open(f, "rb")
    dataDict = pickle.load(batch, encoding = "latin1") # load the batch with pickle
    xtemp = dataDict["data"] # Create temp var
    ytemp = dataDict["labels"]
    xtemp = xtemp.reshape(10000, 3072) # reshape to the proper size
    ys = np.array(ytemp)
    X.append(xtemp) # add to the list
    Y.append(ytemp)

X_train = np.concatenate(X) # Turn into np arrays for training variables
Y_train = np.concatenate(Y)

print(X_train.shape)
print(Y_train.shape)
print(dataDict.keys()) # Just to show the various categories there was
print(type(X_train))
print(type(Y_train))

del xtemp, ytemp, X, Y # Delete the temporary variables and the old lists
# Some code that helped me when I got stuck: https://stackoverflow.com/questions/37512290/reading-cifar10-dataset-in-batches

(50000, 3072)
(50000,)
dict_keys(['batch_label', 'labels', 'data', 'filenames'])
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [26]:
# Now that we have the training variables, it's time to get the test dataset!
# The data is hidden inside a file called test_batch

test_set = os.path.join(cifarDirBatches, "test_batch")
test_batch = open(test_set, "rb")
testDict = pickle.load(test_batch, encoding="latin1")
X_test = testDict["data"]
Y_test = testDict["labels"]
X_test = X_test.reshape(10000, 3072)
Y_test = np.array(Y_test)

print(X_test.shape)
print(Y_test.shape)

(10000, 3072)
(10000,)


In [ ]:
# Now let's try to fit them to a model, SVC seems like a good choice!
# Be wary, it will take a while! Scikit learn doesn't have cuda support so it's really slow! (It took over half 
# hour for mine to train)

if os.exists("SVCCLF.p"):
    svc_clf = pickle.load(open("SVCCLF.p", "rb"))
else:
    svc_clf = SVC()
    svc_clf.fit(X_train, Y_train)
    print("Done fitting!")
    # It worked, let's save the model real quick
    pickle.dump(svc_clf, open("SVCCLF.p", "wb"))